In [1]:
import pandas as pd

In [2]:
df_customers = pd.read_csv(r'C:/Users/Eljan Mammadbayov/OneDrive/Рабочий стол/Python Practice/Euromart Claude/raw/customers.csv')

df_customers

,customer_id,first_name,last_name,country,city,email,signup_date,loyalty_status,age
0,CUST_1001,Gabriel,Simon,France,Marseille,gabriel.simon@gmail.com,10/5/2022,Silver,55.0
1,CUST_1002,Hannah,Schneider,Germany,Berlin,NaN,1/13/2025,Gold,70.0
2,CUST_1003,Jules,Martin,France,Lille,jules.martin@yahoo.com,9/8/2022,NaN,65.0
3,CUST_1004,Jules,Michel,France,Marseille,jules.michel@icloud.com,10/19/2024,NaN,34.0
4,CUST_1005,Emma,Mertens,Luxembourg,Luxembourg City,emma.mertens@yahoo.com,9/2/2024,NaN,51.0
...,...,...,...,...,...,...,...,...,...
1215,CUST_2216,Mia,Fischer,Germany,Berlin,mia.fischer@icloud.com,8/7/2023,Silver,30.0
1216,CUST_2217,Daan,Visser,Netherlands,Utrecht,daan.visser@yahoo.com,5/18/2024,Platinum,48.0
1217,CUST_2218,Louis,Martin,France,Nice,louis.martin@gmail.com,4/10/2024,Platinum,NaN
1218,CUST_2219,Marie,Dubois,Luxembourg,Esch-sur-Alzette,marie.dubois@yahoo.com,NaN,Silver,52.0


In [3]:
df_customers.isnull().sum()

customer_id         0
first_name          0
last_name           0
country             0
city                0
email              41
signup_date        50
loyalty_status    524
age                43
dtype: int64

In [4]:
#Cleaning will be based on data profiling performed on SQL. Issues have already been noted.
#1 First, I will fix malformed emails (emails missing @ and emails in caps)

In [5]:
def fix_email_format(email):
    """Fix common email issues"""
    if pd.isna(email):
        return None
    
    email = str(email).strip()
    
    # Fix missing @ symbol (e.g., "namegmail.com" → "name@gmail.com")
    if '@' not in email and '.' in email:
        # Find common domains
        domains = ['gmail.com', 'outlook.com', 'yahoo.com', 'hotmail.com', 'icloud.com']
        for domain in domains:
            if domain in email.lower():
                email = email.lower().replace(domain, f'@{domain}')
                break
    
    # Standardize to lowercase
    email = email.lower()

    # Basic validation
    # Confirming that email domain has the right structure (gmail.com, yahoo.com). If not, return none
    if '@' in email and '.' in email.split('@')[1]:
        return email
    else:
        return None  # Invalid email

# Apply fix
df_customers['email_original'] = df_customers['email']  # Backup
df_customers['email'] = df_customers['email'].apply(fix_email_format)

In [6]:
df_customers[['email']].head(25)

,email
0,gabriel.simon@gmail.com
1,None
2,jules.martin@yahoo.com
3,jules.michel@icloud.com
4,emma.mertens@yahoo.com
5,anna.visser@gmail.com
6,sophie.jansen@gmail.com
7,lucas.vandenberg@outlook.com
8,liam.visser@hotmail.com
9,gabriel.michel@icloud.com


In [7]:
#2 Second, merging customers with duplicate emails and keeping their earliest sign up data.

# Step 1: Identify duplicates by email
duplicate_emails = df_customers[df_customers['email'].notna()].groupby('email').filter(lambda x: len(x) > 1)

print(f"Found {len(duplicate_emails)} customer records with duplicate emails")
print(f"Affecting {duplicate_emails['email'].nunique()} unique email addresses")

# Step 2: For each duplicate email, keep the earliest customer
def merge_duplicate_customers(df):
    """
    For duplicate emails:
    1. Keep the customer with earliest signup_date
    2. Create mapping table for order reassignment
    3. Remove duplicate customer records
    """
    
    # Create mapping: duplicate_id → primary_id
    customer_mapping = {}
    
    # Group by email
    for email, group in df.groupby('email'):
        if len(group) > 1 and pd.notna(email):
            # Sort by signup_date (earliest first), then by customer_id
            sorted_group = group.sort_values(['signup_date', 'customer_id'], na_position='last')
            
            primary_id = sorted_group.iloc[0]['customer_id']
            duplicate_ids = sorted_group.iloc[1:]['customer_id'].tolist()
            
            # Map duplicates to primary
            for dup_id in duplicate_ids:
                customer_mapping[dup_id] = primary_id
            
            print(f"Email: {email}")
            print(f"  Primary: {primary_id}")
            print(f"  Duplicates: {duplicate_ids}")
    
    # Keep only primary customers (remove duplicates)
    customers_to_remove = list(customer_mapping.keys())
    df_clean = df[~df['customer_id'].isin(customers_to_remove)].copy()
    
    print(f"\n✓ Removed {len(customers_to_remove)} duplicate customers")
    print(f"✓ Kept {len(df_clean)} unique customers")
    
    return df_clean, customer_mapping

df_customers_clean, customer_id_mapping = merge_duplicate_customers(df_customers)

# Save mapping for later (to update orders table)
pd.DataFrame(list(customer_id_mapping.items()), 
             columns=['old_customer_id', 'new_customer_id']).to_csv('C:/Users/Eljan Mammadbayov/OneDrive/Рабочий стол/Python Practice/Euromart Claude/customer_id_mapping.csv', index=False)

Found 750 customer records with duplicate emails
Affecting 313 unique email addresses
Email: alice.bernard@yahoo.com
  Primary: CUST_1439
  Duplicates: ['CUST_2137']
Email: alice.dubois@gmail.com
  Primary: CUST_1715
  Duplicates: ['CUST_2220', 'CUST_1646', 'CUST_1837']
Email: alice.laurent@hotmail.com
  Primary: CUST_1280
  Duplicates: ['CUST_1775']
Email: alice.lefebvre@gmail.com
  Primary: CUST_1661
  Duplicates: ['CUST_1989', 'CUST_1474']
Email: alice.lefebvre@outlook.com
  Primary: CUST_1791
  Duplicates: ['CUST_1379', 'CUST_1321', 'CUST_1391']
Email: alice.lefebvre@yahoo.com
  Primary: CUST_1575
  Duplicates: ['CUST_2167', 'CUST_2180', 'CUST_2129']
Email: alice.martin@hotmail.com
  Primary: CUST_1882
  Duplicates: ['CUST_2131', 'CUST_2061']
Email: alice.martin@icloud.com
  Primary: CUST_1347
  Duplicates: ['CUST_1078', 'CUST_1042']
Email: alice.martin@yahoo.com
  Primary: CUST_1477
  Duplicates: ['CUST_1116']
Email: anna.bakker@outlook.com
  Primary: CUST_1718
  Duplicates: ['CUS

In [8]:
#3 Third, I will handle null values

# Handle loyalty_status (21.6% NULL)
df_customers_clean['loyalty_status'] = df_customers_clean['loyalty_status'].fillna('Unclassified')

# Handle age (3.5% NULL) - Impute with median by country
df_customers_clean['age'] = df_customers_clean.groupby('country')['age'].transform(
    lambda x: x.fillna(x.median())
)

# If still NULL (edge case), use overall median
# df_customers_clean['age'] = df_customers_clean['age'].fillna(df_customers_clean['age'].median())

# Handle signup_date (4.1% NULL) - Flag as unknown
df_customers_clean['signup_date_missing'] = df_customers_clean['signup_date'].isna()
# Keep as NULL or impute with earliest date in dataset
# df_customers_clean['signup_date'] = df_customers_clean['signup_date'].fillna('2021-01-01')

In [9]:
df_customers_clean['loyalty_status'].head(30)

0           Silver
1             Gold
2     Unclassified
3     Unclassified
8     Unclassified
9     Unclassified
10        Platinum
11    Unclassified
12          Silver
14            Gold
15    Unclassified
16        Platinum
20    Unclassified
21          Silver
24    Unclassified
25    Unclassified
27    Unclassified
30          Silver
32    Unclassified
35        Platinum
37          Silver
39        Platinum
40        Platinum
44            Gold
45        Platinum
46        Platinum
47        Platinum
50    Unclassified
51          Silver
52    Unclassified
Name: loyalty_status, dtype: object

In [10]:
df_customers_clean['age'].head(30)

0     55.0
1     70.0
2     65.0
3     34.0
8     52.0
9     33.0
10    68.0
11    27.0
12    67.0
14    69.0
15    31.0
16    49.0
20    66.0
21    54.0
24    40.0
25    64.0
27    70.0
30    53.0
32    60.0
35    32.0
37    39.0
39    44.0
40    28.0
44    58.0
45    55.0
46    45.0
47    25.0
50    54.0
51    68.0
52    38.0
Name: age, dtype: float64

In [11]:
df_customers_clean[df_customers_clean['signup_date_missing']]

,customer_id,first_name,last_name,country,city,email,signup_date,loyalty_status,age,email_original,signup_date_missing
220,CUST_1221,Marie,Dubois,Luxembourg,Esch-sur-Alzette,marie.dubois@yahoo.com,NaN,Silver,52.0,marie.dubois@yahoo.com,True
318,CUST_1319,Lars,Jacobs,Belgium,Ghent,lars.jacobs@gmail.com,NaN,Unclassified,25.0,lars.jacobs@gmail.com,True
356,CUST_1357,Noah,Mertens,Belgium,Bruges,noah.mertens@outlook.com,NaN,Unclassified,46.0,noah.mertensoutlook.com,True
413,CUST_1414,Lucas,Mertens,Belgium,Ghent,lucas.mertens@icloud.com,NaN,Unclassified,67.0,lucas.mertens@icloud.com,True
446,CUST_1447,Louis,Michel,France,Marseille,louis.michel@hotmail.com,NaN,Gold,40.0,louis.michel@hotmail.com,True
477,CUST_1478,Paul,Weber,Germany,Berlin,paul.weber@yahoo.com,NaN,Gold,54.0,paul.weber@yahoo.com,True
513,CUST_1514,Gabriel,Dubois,France,Nice,gabriel.dubois@gmail.com,NaN,Unclassified,29.0,gabriel.dubois@gmail.com,True
559,CUST_1560,Julia,van den Berg,Netherlands,Eindhoven,julia.vandenberg@outlook.com,NaN,Unclassified,32.0,JULIA.VANDENBERG@OUTLOOK.COM,True
576,CUST_1577,Noah,Mertens,Luxembourg,Esch-sur-Alzette,noah.mertens@gmail.com,NaN,Platinum,67.0,NOAH.MERTENS@GMAIL.COM,True
658,CUST_1659,Chloé,Peeters,Luxembourg,Esch-sur-Alzette,chloé.peeters@hotmail.com,NaN,Platinum,47.0,chloé.peeters@hotmail.com,True


In [12]:
#4 Standardizing city and country columns just in case (there should be no problems there)

# Standardize country names (proper case)
df_customers_clean['country'] = df_customers_clean['country'].str.title()

# Standardize city names
df_customers_clean['city'] = df_customers_clean['city'].str.title()

# Final validation
print("\n✓ Customers Table Cleaned!")
print(f"Original rows: {len(df_customers)}")
print(f"Cleaned rows:  {len(df_customers_clean)}")
print(f"Rows removed:  {len(df_customers) - len(df_customers_clean)}")


✓ Customers Table Cleaned!
Original rows: 1220
Cleaned rows:  783
Rows removed:  437


In [13]:
# Save cleaned customers
df_customers_clean.to_csv('C:/Users/Eljan Mammadbayov/OneDrive/Рабочий стол/Python Practice/Euromart Claude/customers_cleaned.csv', index=False)
print("✓ Saved: C:/Users/Eljan Mammadbayov/OneDrive/Рабочий стол/Python Practice/Euromart Claude/customers_cleaned.csv")

✓ Saved: C:/Users/Eljan Mammadbayov/OneDrive/Рабочий стол/Python Practice/Euromart Claude/customers_cleaned.csv
